# Day 21

In [1]:
import operator

## Part 1

In [2]:
ops = {" + ": operator.add, " - ": operator.sub, " / ": operator.floordiv, " * ": operator.mul, " = ": operator.eq}

In [3]:
def parse_line(line:str, monkeys: dict)->dict:
    key = line.strip('\n').split(': ')[0]
    val = line.strip('\n').split(': ')[1]
    try:
        monkeys[key] = int(val)
    except ValueError:
        monkeys[key] = val
    return monkeys

In [4]:
def part1(monkeys:dict)->int:
    # recursively iterate through the dict until we can compute root:
    flag = True
    while flag:
        flag = False
        for key in monkeys.keys():
            if isinstance(monkeys[key],int):
                continue
            flag = True 
            val = monkeys[key]
            for op in ops.keys():
                if op in val:
                    left, right = val.split(op)
                    try:
                        monkeys[key] = int(ops[op](monkeys[left],monkeys[right]))
                    except ValueError:
                        break
                    except TypeError:
                        # One value is known, the other isn't
                        for c in [left,right]:
                            if isinstance(monkeys[c],int):
                                continue
                            try:
                                monkeys[c] = int(monkeys[c])
                            except ValueError:
                                break
    return(monkeys['root'])

In [5]:
with open('test_day21.txt') as input_text:
    monkeys = {}
    for line in input_text:
        monkeys = parse_line(line,monkeys)
    assert(part1(monkeys)==152)

In [6]:
with open('input_day21.txt') as input_text:
    monkeys = {}
    for line in input_text:
        monkeys = parse_line(line,monkeys)
    print(part1(monkeys))

31017034894002


## Part 2

In [15]:
reverse_op = {" + ":" - "," - ":" + "," * ":" / "," / ":" * "}

In [23]:
def parse_value(val:str)->list[str]:
    for op in ops:
        if op in val:
            new_val = val.split(op)
            return [new_val[0],op,new_val[1]]

In [78]:
def parse_line_part2(line:str, monkeys: dict)->dict:
    key = line.strip('\n').split(': ')[0]
    val = line.strip('\n').split(': ')[1]
    if 'root' in key:
        for op in ops:
            if op in val:
                new_val = val.replace(op,' = ').split(' = ')
                monkeys['root'] = [new_val[0],' = ',new_val[1]]
                return monkeys
    try:
        monkeys[key] = int(val)
    except ValueError:
        monkeys[key] = parse_value(val)
    return monkeys

In [81]:
# find side of equation with 'humn' in it.  We can evaluate the other side, and we will "undo" the 'humn' side
def humn_side_of_equation(k):
    val = monkeys[k]
    if isinstance(val,int):
        return False
    elif "humn" in val:
        return True
    else:
        return humn_side_of_equation(val[0]) or humn_side_of_equation(val[2])

In [82]:
def evaluate(k, rpn=False):
    val = monkeys[k]
    if isinstance(val,int):
        if k=="humn" and rpn:
            return ["x"]
        return [val] if rpn else val
    else:
        if rpn:
            param1 = [*evaluate(val[0],rpn)]
            param2 = [*evaluate(val[2],rpn)]
            if len(param1)==len(param2) and len(param1)==1:
                if isinstance(param1[0],int) and isinstance(param2[0],int):
                    return [ops[val[1]](param1[0],param2[0])]
            return [param1,val[1],param2]
        return (ops[val[1]](evaluate(val[0]),evaluate(val[2])))

In [90]:
#find side w/ humn branch
def part2(monkeys:dict)->int:
    k = monkeys["root"]
    humn_branch,nonhumn_branch = (k[0],k[2]) if humn_side_of_equation(k[0]) else (k[2],k[0])
    rhs = evaluate(nonhumn_branch)

    #now we need the evaluation of humn_branch to be equal to nonhumn_branch
    unpacked_equation = evaluate(humn_branch,True)
    # #work down to "x"
    while True:
        if unpacked_equation==["x"]: break
        i1,op,i2 = unpacked_equation
        if len(i1)==1 and isinstance(i1[0],int):
            i = i1[0]
            if op in [' + ', ' - ']:
                o = " - "
            else: o = " / "
            if op in [' + ',' * ']:
                rhs = ops[o](rhs,i)
            else:
                rhs = ops[o](i, rhs)
            unpacked_equation = i2
        elif len(i2)==1 and isinstance(i2[0],int):
            i = i2[0]
            rhs = ops[reverse_op[op]](rhs,i)
            unpacked_equation = i1
    print(rhs)
    return(rhs)

In [91]:
with open('test_day21.txt') as input_text:
    monkeys = {}
    for line in input_text:
        monkeys = parse_line_part2(line,monkeys)
    assert(part2(monkeys)==301)

301


In [93]:
with open('input_day21.txt') as input_text:
    monkeys = {}
    for line in input_text:
        monkeys = parse_line_part2(line,monkeys)
    part2(monkeys)

3555057453229
